In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from  helper import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
item_train, user_train, y_train, item_features, user_features, item_vecs, movie_dict, = load_data()

num_user_features = user_train.shape[1] - 3  # remove userid, rating count and ave rating during training
num_item_features = item_train.shape[1] - 1  # remove movie id at train time
uvs = 3  # user genre vector start
ivs = 3  # item genre vector start
u_s = 3  # start of columns to use in training, user
i_s = 1  # start of columns to use in training, items

In [3]:
item_train_unscaled = item_train
user_train_unscaled = user_train
y_train_unscaled    = y_train

scalerItem = StandardScaler()
scalerItem.fit(item_train)
item_train = scalerItem.transform(item_train)

scalerUser = StandardScaler()
scalerUser.fit(user_train)
user_train = scalerUser.transform(user_train)

scalerTarget = MinMaxScaler((-1, 1))
scalerTarget.fit(y_train.reshape(-1, 1))
y_train = scalerTarget.transform(y_train.reshape(-1, 1))

item_train, item_test = train_test_split(item_train, train_size=0.80, shuffle=True, random_state=1)
user_train, user_test = train_test_split(user_train, train_size=0.80, shuffle=True, random_state=1)
y_train, y_test       = train_test_split(y_train,    train_size=0.80, shuffle=True, random_state=1)

In [4]:
num_user_features = 14
num_item_features = 16
num_outputs = 32
tf.random.set_seed(1)
# User network
user_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs),
])

# Item network
item_NN = tf.keras.models.Sequential([
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_outputs),
])

# Create the user input and process it through the user network
import tensorflow as tf

# Define the user input and process it through the user network
input_user = tf.keras.layers.Input(shape=(num_user_features,))
vu = user_NN(input_user)
vu = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vu)

# Define the item input and process it through the item network
input_item = tf.keras.layers.Input(shape=(num_item_features,))
vm = item_NN(input_item)
vm = tf.keras.layers.Lambda(lambda x: tf.linalg.l2_normalize(x, axis=1))(vm)


# Compute the dot product of the two vectors vu and vm
output = tf.keras.layers.Dot(axes=1)([vu, vm])

# Specify the inputs and output of the model
model = tf.keras.Model([input_user, input_item], output)

In [5]:
tf.random.set_seed(1)
cost_fn = tf.keras.losses.MeanSquaredError()
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer=opt,
              loss=cost_fn)


tf.random.set_seed(1)
model.fit([user_train[:, u_s:], item_train[:, i_s:]], y_train, epochs=30)

Epoch 1/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step - loss: 0.1299
Epoch 2/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1173
Epoch 3/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1150
Epoch 4/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1138
Epoch 5/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1128
Epoch 6/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1118
Epoch 7/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1110
Epoch 8/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1101
Epoch 9/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1093
Epoch 10/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1082
Epoch 11/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1069
Epoch 12/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.1060
Epoch 13/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.1054
Epoch 14/30
1455/1455 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - loss: 0.1047
Epoch 15/30
1455/1455 ━━━━━━━

In [6]:
new_user_id = 5000
new_rating_ave = 0.0
new_action = 0.0
new_adventure = 5.0
new_animation = 0.0
new_childrens = 0.0
new_comedy = 0.0
new_crime = 0.0
new_documentary = 0.0
new_drama = 5.0
new_fantasy = 5.0
new_horror = 0.0
new_mystery = 5.0
new_romance = 0.0
new_scifi = 0.0
new_thriller = 0.0
new_rating_count = 3

user_vec = np.array([[new_user_id, new_rating_count, new_rating_ave,
                      new_action, new_adventure, new_animation, new_childrens,
                      new_comedy, new_crime, new_documentary,
                      new_drama, new_fantasy, new_horror, new_mystery,
                      new_romance, new_scifi, new_thriller]])

In [7]:
def movies_predict(user_vec):
    user_vecs = gen_user_vecs(user_vec,len(item_vecs))
    suser_vecs = scalerUser.transform(user_vecs)
    sitem_vecs = scalerItem.transform(item_vecs)

# make a prediction
    y_p = model.predict([suser_vecs[:, u_s:], sitem_vecs[:, i_s:]])

# unscale y prediction 
    y_pu = scalerTarget.inverse_transform(y_p)

# sort the results, highest prediction first
    sorted_index = np.argsort(-y_pu,axis=0).reshape(-1).tolist()  #negate to get largest rating first
    sorted_ypu   = y_pu[sorted_index]
    sorted_items = item_vecs[sorted_index] 

    return print_pred_movies(sorted_ypu, sorted_items, movie_dict, maxcount = 10)

In [8]:
movies_predict(user_vec)

59/59 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


[['title', 'genres'],
 ['Mary and Max (2009)', 'Animation|Comedy|Drama'],
 ["Amelie (Fabuleux destin d'AmÃ©lie Poulain, Le) (2001)", 'Comedy|Romance'],
 ['Persepolis (2007)', 'Animation|Drama'],
 ['Lord of the Rings: The Return of the King, The (2003)',
  'Action|Adventure|Drama|Fantasy'],
 ['Spring, Summer, Fall, Winter... and Spring (Bom yeoreum gaeul gyeoul geurigo bom) (2003)',
  'Drama'],
 ["Howl's Moving Castle (Hauru no ugoku shiro) (2004)",
  'Adventure|Animation|Fantasy|Romance'],
 ['City of God (Cidade de Deus) (2002)',
  'Action|Adventure|Crime|Drama|Thriller']]